# Preraring the data

Before moving on to the exercise, we will have to conduct the exact same data preparation.

## Importing packages

Firstly, let's include all imports of libraries we want in a single cell for convenience:

In [1]:
import random

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

RANDOM_SEED = 42
TORCH_GEN_SEED = 2147483647

## Loading the data

Next step is to load the names:

In [2]:
# Opening the dataset with names and reading its content into a variable
words = open("../names.txt", "r").read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

## Building the vocabulary and creating character-number mappings

Now we create the vocabulary as well as mappings from character to its identifier and backwards:

In [3]:
# Retrieving a set of unique letters
chars = sorted(list(set(''.join(words))))

# Creating a mapping from a letter to an id
char2id = {s: i+1 for i, s in enumerate(chars)}
# Adding the start_of_word/end_of_word token => "."
char2id['.'] = 0

# Creating a mapping from an id to letter
id2char = {i: s for s, i in char2id.items()}

# Computing the size of the vocabulary
vocab_size = len(id2char)

# Displaying the mappings and vocab size
print("Character -> Identifier:")
print(char2id)
print()
print("Identifier -> Character:")
print(id2char)
print()
print(f"Vocabulary size: {vocab_size}")

Character -> Identifier:
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}

Identifier -> Character:
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}

Vocabulary size: 27


## Splitting the data

In [4]:
# Setting the random seed and reshuffling names
random.seed(RANDOM_SEED)
random.shuffle(words)

# Determining cutoff points for 10% dev and 10% test
cutoff_train = int(0.8*len(words))
cutoff_dev = int(0.9*len(words))

print(f"Training set: {0}-{cutoff_train-1:,}")
print(f"Development set: {cutoff_train:,}-{cutoff_dev-1:,}")
print(f"Testing set: {cutoff_dev:,}-{len(words)-1:,}")

print(f"\nTotal words: {len(words):,}")

Training set: 0-25,625
Development set: 25,626-28,828
Testing set: 28,829-32,032

Total words: 32,033


In [5]:
# Allocating shuffled words into three sets
words_train = words[:cutoff_train]
words_dev = words[cutoff_train:cutoff_dev]
words_test = words[cutoff_dev:]

print(f"Training set examples: {len(words_train):,} ({len(words_train)/len(words):.0%})")
print(f"Development set examples: {len(words_dev):,} ({len(words_dev)/len(words):.0%})")
print(f"Testing set examples: {len(words_test):,} ({len(words_test)/len(words):.0%})")

Training set examples: 25,626 (80%)
Development set examples: 3,203 (10%)
Testing set examples: 3,204 (10%)


In [6]:
def build_dataset(words, block_size):
    """Creates a dataset of encoded characters."""
    # Preallocating lists for dataset
    X, Y = [], []
    for word in words:
        # Creating a starting examples depending on block size
        context = [0] * block_size
        # Iterating through entire word with end of word token
        for char in word + '.':
            index = char2id[char]
            X.append(context)
            Y.append(index)
            # Adding the character index and shifting
            context = context[1:] + [index]
    
    # Casting as PyTorch tensors
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return X, Y

In [7]:
# Setting the block size (number of character to use to predict the next one)
block_size = 3

# Building the datasets for three sets
X_train, Y_train = build_dataset(
    words=words_train, block_size=block_size,
)
X_dev, Y_dev = build_dataset(
    words=words_dev, block_size=block_size,
)
X_test, Y_test = build_dataset(
    words=words_test, block_size=block_size,
)

# One-epoch forward/backward pass runs

Next, we will run the training loop for one iteration in order to be able to collect the gradients computed using `backward` method from `torch` that will be later used to doing gradient comparisons.

## Initialization

The first obvious step is to initialize all weights of the neural net. In the following cell we fix: 

* dimensionality of the space into which the characters fed into the net will be mapped (`n_embd`);
* number of neurons of which the hidden layer will be composed (`n_hidden`);
* random number generator (`generator`).

In [8]:
# Dimensionality of the character embedding vectors
n_embd = 10

# number of neurons in the hidden layer of the MLP
n_hidden = 64

# Setting up the random numbers generator (for reproducibility)
generator = torch.Generator().manual_seed(TORCH_GEN_SEED)

Next, we set up the MLP weights themselves. It should be noted that the parameters are initialized in such a way that the comparison of the backward pass implementations becomes more accurate (initialization by zeros can mask the incorrect backpropagation implementation).

In [9]:
# Initializing the character embedding matrix
C  = torch.randn((vocab_size, n_embd), generator=generator)

# LAYER 1 WEIGHTS (Kaiming initialization)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=generator)
W1 *= (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=generator) * 0.1

# LAYER 2 WEIGHTS
W2 = torch.randn((n_hidden, vocab_size), generator=generator) * 0.1
b2 = torch.randn(vocab_size, generator=generator) * 0.1

# BATCHNORM LAYER PARAMETERS
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

Next, we combine all parameters to be trained together, compute its total number and enable gradient computation:

In [10]:
# Putting all trainable parameters together
parameters = [C, W1, b1, W2, b2, bngain, bnbias]

# Computing the number of parameters
num_params = sum(p.nelement() for p in parameters)
print(f"Number of trainable parameters: {num_params:,}") # number of parameters in total

# Making all parameters trainable (enabling gradient computation)
for p in parameters:
    p.requires_grad = True

Number of trainable parameters: 4,137


## Selecting minibatch

In [11]:
# Setting batch size
batch_size = 32
n = batch_size

# Constructing a minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=generator)
Xb, Yb = X_train[ix], Y_train[ix]

## Forward pass

In [12]:
# Embedding the characters into vectors
emb = C[Xb]
# Concatenating the vectors
embcat = emb.view(emb.shape[0], -1)

#################################################

# LINEAR LAYER 1

# Hidden layer pre-activation
hprebn = embcat @ W1 + b1

#################################################

# BATCHNORM LAYER

bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

#################################################

# TANH LAYER
h = torch.tanh(hpreact)

#################################################

# LINEAR LAYER 2

# Output layer
logits = h @ W2 + b2 # output layer

#################################################

# CROSS-ENTROPY LOSS

logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

## Backward pass (Pytorch)

In [13]:
# Zeroing all gradients
for p in parameters:
    p.grad = None
    
# Computing gradients for each element of the computational graph
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()

loss

tensor(3.3236, grad_fn=<NegBackward0>)

# Exercise 1: Manual backpropagation

Now we will have to manually backpropagate throught the net and compare the computed gradients with those computed by Pytorch. We will use `cmp` function for being able to make such comparison which is defined below:

In [14]:
def cmp(s, dt, t):
    """Compares manual gradients to Pytorch gradients."""
    # Returning True if all gradients are exactly equal
    ex = torch.all(dt == t.grad).item()
    # Returning True if all gradients are approximately equal
    app = torch.allclose(dt, t.grad)
    # Computing the maximum difference between gradients
    maxdiff = (dt - t.grad).abs().max().item()
    # Printing the comparison information
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

Now we will go layer by layer backwards computing gradients and comparing them to those given by Pytorch:

1. Cross-entropy layer
2. Linear layer 2
3. Tanh layer
4. Linear layer 1

## Cross-entropy layer

### `dlogprobs`

The first element we have to backpropagate through starts at the end of the forward pass:

```python
loss = -logprobs[range(n), Yb].mean()
```

In this case we need to find the derivative of `loss` with respect to all of the elements of `logprobs`. Let's consider the shape of `logprobs`:

In [15]:
logprobs.shape

torch.Size([32, 27])

What we basically do when computing the loss is that we just go across the rows of `logprobs` and pluck out a specific value from each row. `Yb` represents all the correct answers from the minibatch and we use them to select the elements of logprobs.

In [16]:
logprobs

tensor([[-2.5977, -2.5081, -3.8608, -3.0493, -3.9454, -2.5466, -3.7662, -3.2970,
         -3.8562, -3.4023, -3.2474, -3.3003, -3.2509, -3.5452, -3.3283, -4.2301,
         -4.6508, -3.9057, -4.1790, -2.9146, -2.9798, -3.8614, -3.5935, -2.6896,
         -2.9245, -3.5907, -3.8202],
        [-2.8943, -2.9508, -2.3043, -3.0001, -3.3874, -3.4851, -4.0099, -3.1080,
         -3.9075, -3.6021, -2.9709, -3.1078, -3.0784, -3.5860, -3.0310, -3.1503,
         -3.6338, -3.9293, -3.9368, -3.3053, -3.9206, -3.8271, -4.2068, -2.7865,
         -3.7794, -3.2778, -3.6353],
        [-3.8616, -3.8408, -4.1239, -4.4752, -3.8493, -3.1846, -2.8557, -2.7162,
         -2.8016, -3.4985, -3.8457, -3.3742, -3.1721, -3.0913, -3.7645, -3.5857,
         -4.2611, -3.3480, -3.5715, -2.1515, -2.7222, -3.3030, -3.1284, -3.3161,
         -3.4061, -3.8846, -3.5488],
        [-3.3517, -3.7365, -3.0457, -3.0286, -2.8694, -3.6259, -3.0988, -3.1669,
         -3.0560, -3.8834, -3.2423, -3.6165, -3.3994, -3.2630, -2.8299, -2.6186

In [17]:
Yb

tensor([ 8, 14, 15, 22,  0, 19,  9, 14,  5,  1, 20,  3,  8, 14, 12,  0, 11,  0,
        26,  9, 25,  0,  1,  1,  7, 18,  9,  3,  5,  9,  0, 18])

In [18]:
logprobs[range(n), Yb]

tensor([-3.8562, -3.0310, -3.5857, -3.1667, -4.0211, -3.4529, -3.1243, -4.0221,
        -3.2409, -4.3274, -3.0983, -1.7218, -2.8576, -2.9392, -2.9631, -3.1084,
        -3.8263, -2.9020, -3.5922, -3.3758, -2.8667, -2.9059, -4.3446, -4.0570,
        -3.5202, -2.9412, -2.9492, -3.9462, -2.8478, -3.3978, -3.1882, -3.1773],
       grad_fn=<IndexBackward0>)

In [19]:
-logprobs[range(n), Yb].mean()

tensor(3.3236, grad_fn=<NegBackward0>)

In order to be able to define what the derivative of this function would look like, we can look at a simpler example. Let's rewrite the loss function in a much simpler form where we have just 3 log-probabilities instead of 32:

$$loss = -\frac{a + b + c}{3}$$

Now the derivatives:

$$\frac{\partial{loss}}{\partial{a}} = -\frac{1}{3}$$

$$\frac{\partial{loss}}{\partial{b}} = -\frac{1}{3}$$

$$\frac{\partial{loss}}{\partial{c}} = -\frac{1}{3}$$

In other words, if we interpolate the results of the above equations to our case, then basically, given that we have `n` instead of `3`, the derivative will take the value of $-\frac{1}{n}$ in all the places that have been plucked out. As far as the other elements are concerned, gradients for these will be zeros, since these simply do not participate in the loss calculation. Hence, changing these elements will not change the loss function in any way so their respective derivatives are zero.

In [20]:
# Computing "dloss/dlogprobs"
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n

# Comparing the gradient computed
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


### `dprobs`

We go one step back and this is the equation we need to backpropagate through:

```python
logprobs = probs.log()
```

In [21]:
logprobs.shape, probs.shape

(torch.Size([32, 27]), torch.Size([32, 27]))

The shapes are the equal to each other so we can just take a simple derivative from calculus (not forgetting the chain rule):

In [22]:
# Computing "dloss/dprobs"
dprobs = (1.0 / probs) * dlogprobs

# Comparing the gradient computed
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


### `dcounts_sum_inv`

```python
probs = counts * counts_sum_inv
```

In [23]:
probs.shape, counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 27]), torch.Size([32, 1]))

In [24]:
dprobs.shape

torch.Size([32, 27])

After considering the shapes above, we see that the dimensions are different for the elements of the multiplication. When encountering such situations, `torch` implements a *broadcasting*: for instance, here `counts_sum_inv` will get replicated across the columns of `counts` 27 times, hence enabling element-wise multiplication. 

We can demonstrate it in a simpler example. Suppose we also have 3 matrices that are connected like so:

$$C = A b$$

Matrices on the right-hand side look as follows (take for instance 3 dimensions):

$$A = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}$$

$$b = \begin{bmatrix}
b_{1}\\
b_{2}\\
b_{3}
\end{bmatrix}$$

Then, applying broadcasting will result in the following:

$$C = \begin{bmatrix}
a_{11} b_1 & a_{12} b_1 & a_{13} b_1\\
a_{21} b_2 & a_{22} b_2 & a_{23} b_2\\
a_{31} b_3 & a_{32} b_3 & a_{33} b_3
\end{bmatrix}$$

We basically replicated $b$ vector across the columns of $A$. Taking $\frac{\partial{probs}}{\partial{counts\_sum\_inv}}$ results in basically taking elements of `count` (multiplied by `dprobs` from the chain rule) and summing them up across the columns (since for instance $b_1$ occurs in all columns and these contributions should be accumulated). 

In [25]:
# Computing "dloss/dcounts_sum_inv"
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)

# Comparing the gradient computed
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


### `dcounts`, `dcounts_sum`

Finding `dcounts` from the code below can be tricky, since `counts` occurs in two different nodes and as a result such contributions should be summed up.

```python
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
```

#### `dcounts` (first node)

Firstly, we will consider the first branch where `counts` occurs:

```python
probs = counts * counts_sum_inv
```

In [26]:
probs.shape, counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 27]), torch.Size([32, 1]))

In [27]:
dprobs.shape

torch.Size([32, 27])

Using the result of the calculations above we can just multiply `counts_sum_inv` by the `dprobs` from the chain rule to obtain the matrix with the correct dimensions.

In [28]:
# Computing "dloss/dcounts" (Node 1)
dcounts = counts_sum_inv * dprobs

# Comparing the gradient computed
cmp('counts', dcounts, counts)

counts          | exact: False | approximate: False | maxdiff: 0.005585933104157448


We can see that the computed gradient is actually incorrect, since we have only considered one node.

#### `dcounts_sum`

```python
counts_sum_inv = counts_sum**-1
```

In [29]:
counts_sum_inv.shape, counts_sum.shape

(torch.Size([32, 1]), torch.Size([32, 1]))

Shapes are indentical so there is nothing difficult to do:

In [30]:
# Computing "dloss/dcounts_sum"
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv

# Comparing the gradient computed
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


#### `dcounts` (second node)

Now we go on to consider the second node through which `counts` flows:

```python
counts_sum = counts.sum(1, keepdims=True)
```

In [31]:
counts_sum.shape, counts.shape

(torch.Size([32, 1]), torch.Size([32, 27]))

In [32]:
dcounts_sum.shape

torch.Size([32, 1])

Let's use a simpler example and depict what this line of code achieves:

$$A = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}$$

$$\Downarrow$$

$$A^{sum} = \begin{bmatrix}
a_{11} + a_{12} + a_{13}\\
a_{21} + a_{22} + a_{23}\\
a_{31} + a_{32} + a_{33}
\end{bmatrix}$$

We basically take the elements of `counts` and sum the elements across each column. Hence, when taking the derivative across each parameter we will get a matrix of ones to be multiplied with `dcounts_sum` from the chain rule:

In [33]:
# Accumulating "dloss/dcounts" from Node 2
dcounts += torch.ones_like(counts) * dcounts_sum

# Comparing the gradient computed
cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


### `dnorm_logits`

```python
counts = norm_logits.exp()
```

In [34]:
counts.shape, norm_logits.shape

(torch.Size([32, 27]), torch.Size([32, 27]))

The shapes are the same so we can just take a simple derivative. Since the derivative of the exponential function is exponential function, we can just use output of `counts` for the local derivative and `dcounts` as one from the chain rule: 

In [35]:
# Computing "dloss/dnorm_logits"
dnorm_logits = counts * dcounts

# Comparing the gradient computed
cmp('norm_logits', dnorm_logits, norm_logits)

norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0


### `dlogits`, `dlogit_maxes`

Next, we have a similar case where `logits` flows through two nodes:

```python
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
```

#### `dlogits` (first node)

```python
norm_logits = logits - logit_maxes
```

In [36]:
norm_logits.shape, logits.shape, logit_maxes.shape

(torch.Size([32, 27]), torch.Size([32, 27]), torch.Size([32, 1]))

Here the shapes are the same for `norm_logits` and `logits` (`logit_maxes` turns to zero) so we have just take `dnorm_logits` from the chain rule and clone it via `clone()` method:

In [37]:
# Computing "dloss/dlogits" (Node 1)
dlogits = dnorm_logits.clone()

# Comparing the gradient computed
cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 7.916241884231567e-09


In this we have actually obtained the approximate equality. However, we are not done yet, since we still need to consider the second branch.

#### `dlogit_maxes`

```python
norm_logits = logits - logit_maxes
```

In [38]:
norm_logits.shape, logits.shape, logit_maxes.shape

(torch.Size([32, 27]), torch.Size([32, 27]), torch.Size([32, 1]))

We have a similar situation where `torch` implements broadcasting: elements of `logit_maxes` are replicated across the columns of `logits`. Hence, as seen above, we can just sum `dnorm_logits` across the columns and put a negative sign: 

In [39]:
# Computing "dloss/dlogit_maxes"
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)

# Comparing the gradient computed
cmp('logit_maxes', dlogit_maxes, logit_maxes)

logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0


#### `dlogits` (second node)

```python
logit_maxes = logits.max(1, keepdim=True).values
```

In [40]:
logit_maxes.shape, logits.shape

(torch.Size([32, 1]), torch.Size([32, 27]))

Let's again use a simpler example to understand what such an operation entails:

$$A = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}$$

$$\Downarrow$$

$$A^{max} = \begin{bmatrix}
\max{a_{11}, a_{12}, a_{13}}\\
\max{a_{21}, a_{22}, a_{23}}\\
\max{a_{31}, a_{32}, a_{33}}
\end{bmatrix}$$

We basically take the maximum value in each row selecting from the values across the columns. In our case matrix $A$ (`logits`) looks as follows:

In [41]:
logits

tensor([[ 8.5027e-01,  9.3988e-01, -4.1278e-01,  3.9874e-01, -4.9741e-01,
          9.0140e-01, -3.1819e-01,  1.5104e-01, -4.0815e-01,  4.5677e-02,
          2.0063e-01,  1.4772e-01,  1.9711e-01, -9.7188e-02,  1.1970e-01,
         -7.8213e-01, -1.2028e+00, -4.5766e-01, -7.3102e-01,  5.3342e-01,
          4.6820e-01, -4.1344e-01, -1.4548e-01,  7.5842e-01,  5.2346e-01,
         -1.4266e-01, -3.7225e-01],
        [ 3.2419e-01,  2.6773e-01,  9.1420e-01,  2.1835e-01, -1.6888e-01,
         -2.6658e-01, -7.9142e-01,  1.1045e-01, -6.8897e-01, -3.8361e-01,
          2.4757e-01,  1.1067e-01,  1.4007e-01, -3.6754e-01,  1.8753e-01,
          6.8173e-02, -4.1529e-01, -7.1075e-01, -7.1830e-01, -8.6824e-02,
         -7.0208e-01, -6.0863e-01, -9.8831e-01,  4.3196e-01, -5.6090e-01,
         -5.9323e-02, -4.1684e-01],
        [-4.5637e-01, -4.3566e-01, -7.1870e-01, -1.0700e+00, -4.4410e-01,
          2.2059e-01,  5.4951e-01,  6.8897e-01,  6.0356e-01, -9.3277e-02,
         -4.4054e-01,  3.1011e-02,  2.33

The operation of taking the maximum value across columns is equivalent to the following one:

In [42]:
logits.max(1)

torch.return_types.max(
values=tensor([0.9399, 0.9142, 1.2537, 0.7177, 1.7236, 0.8612, 0.7066, 1.3514, 1.1333,
        1.0630, 1.7201, 1.9321, 1.0995, 0.7756, 0.6405, 0.7470, 1.0149, 0.7314,
        1.0995, 0.8617, 0.9042, 0.9328, 1.0995, 1.2289, 1.4748, 0.9446, 1.2910,
        0.9354, 0.9218, 0.8238, 0.9519, 0.8214], grad_fn=<MaxBackward0>),
indices=tensor([ 1,  2, 19, 15, 15, 25, 16,  3,  7,  8, 15,  3, 22,  5,  7,  5,  2,  1,
        22, 19, 15, 19, 22, 22, 23,  5, 22, 20, 16, 15, 24,  2]))

In addition to calculating the maximum values, `max` method also returns the index where this value has been found. We can use this information to compose an array returning one for the maximum value. Basically, ones will denote the place where the gradient is not zero. There are many ways to do this but we will use `one_hot` function:

In [43]:
F.one_hot(logits.max(1).indices, num_classes=logits.shape[1])

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],


In [44]:
# Accumulating "dloss/dlogits" from Node 2
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

# Comparing the gradient computed
cmp('logits', dlogits, logits)

logits          | exact: True  | approximate: True  | maxdiff: 0.0


## Linear layer 2

Now we move on to the linear layer 2. Here is the expression that we need to backrpropagate through:

```python 
logits = h @ W2 + b2
```

In [45]:
logits.shape, h.shape, W2.shape, b2.shape

(torch.Size([32, 27]),
 torch.Size([32, 64]),
 torch.Size([64, 27]),
 torch.Size([27]))

In order to be able to easier define what a derivate looks like with respect to all right-hand side elements of the above expression, let's consider a simpler case:

$$D = A \times B + C$$

$$\Downarrow$$

$$\begin{bmatrix}
d_{11} & d_{12}\\
d_{21} & d_{22}\\
\end{bmatrix} = \begin{bmatrix}
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{bmatrix} \times \begin{bmatrix}
b_{11} & b_{12}\\
b_{21} & b_{22}
\end{bmatrix} + \begin{bmatrix}
c_{1} & c_{2}\\
c_{1} & c_{2}
\end{bmatrix}
$$

In this example, we have the matrix representation of the equation where matrix $C$ (that is `b2`) is getting broadcast across all rows of $A \times B$ (since `b2` is not a column but row vector). Now we will try to derive the derivative with respect to each elements of $A$, $B$ and $C$.

#### `dh`

Firstly, we will find the gradient of `loss` with respect to `h`. In other words, in our toy example, we will be looking for the derivative with respect to elements of $A$. Let's write down what each element of $D$ looks like:

$$d_{11} = a_{11} b_{11} + a_{12} b_{21} + c_1$$
$$d_{12} = a_{11} b_{12} + a_{12} b_{22} + c_2$$
$$d_{21} = a_{21} b_{11} + a_{22} b_{21} + c_1$$
$$d_{22} = a_{21} b_{12} + a_{22} b_{22} + c_2$$

Next, let's start calculating the derivatives of loss function $L$ using the chain rules:

$$\frac{\partial{L}}{\partial{a_{11}}} = \frac{\partial{L}}{\partial{d_{11}}} \frac{\partial{d_{11}}}{\partial{a_{11}}} + \frac{\partial{L}}{\partial{d_{12}}} \frac{\partial{d_{12}}}{\partial{a_{11}}} = \frac{\partial{L}}{\partial{d_{11}}} b_{11} + \frac{\partial{L}}{\partial{d_{12}}} b_{12}$$

$$\frac{\partial{L}}{\partial{a_{12}}} = \frac{\partial{L}}{\partial{d_{11}}} \frac{\partial{d_{11}}}{\partial{a_{12}}} + \frac{\partial{L}}{\partial{d_{12}}} \frac{\partial{d_{12}}}{\partial{a_{12}}} = \frac{\partial{L}}{\partial{d_{11}}} b_{21} + \frac{\partial{L}}{\partial{d_{12}}} b_{22}$$

$$\frac{\partial{L}}{\partial{a_{21}}} = \frac{\partial{L}}{\partial{d_{21}}} \frac{\partial{d_{21}}}{\partial{a_{21}}} + \frac{\partial{L}}{\partial{d_{22}}} \frac{\partial{d_{22}}}{\partial{a_{21}}} = \frac{\partial{L}}{\partial{d_{21}}} b_{11} + \frac{\partial{L}}{\partial{d_{22}}} b_{12}$$

$$\frac{\partial{L}}{\partial{a_{22}}} = \frac{\partial{L}}{\partial{d_{21}}} \frac{\partial{d_{21}}}{\partial{a_{22}}} + \frac{\partial{L}}{\partial{d_{22}}} \frac{\partial{d_{22}}}{\partial{a_{22}}} = \frac{\partial{L}}{\partial{d_{21}}} b_{21} + \frac{\partial{L}}{\partial{d_{22}}} b_{22}$$

It can actually be seen that representing the gradients in a matrix form allows us to rewrite everything down as follows:

$$\begin{bmatrix}
\frac{\partial{L}}{\partial{a_{11}}} & \frac{\partial{L}}{\partial{a_{12}}}\\
\frac{\partial{L}}{\partial{a_{21}}} & \frac{\partial{L}}{\partial{a_{22}}}\\
\end{bmatrix} = \begin{bmatrix}
\frac{\partial{L}}{\partial{d_{11}}} & \frac{\partial{L}}{\partial{d_{12}}}\\
\frac{\partial{L}}{\partial{d_{21}}} & \frac{\partial{L}}{\partial{d_{22}}}\\
\end{bmatrix} \times \begin{bmatrix}
b_{11} & b_{21}\\
b_{12} & b_{22}\\
\end{bmatrix}
$$

$$\Downarrow$$

$$\frac{\partial{L}}{\partial{a}} = \frac{\partial{L}}{\partial{d}} \times b^{T}$$

Now, let's use this formula to compute the gradient:

In [46]:
# Computing "dloss/dh"
dh = dlogits @ W2.T

# Comparing the gradient computed
cmp('h', dh, h)

h               | exact: True  | approximate: True  | maxdiff: 0.0


#### `dW2`

We can do the exact same with elements of $B$ matrix (gradient of `loss` with respect to `W2`):

$$d_{11} = a_{11} b_{11} + a_{12} b_{21} + c_1$$
$$d_{12} = a_{11} b_{12} + a_{12} b_{22} + c_2$$
$$d_{21} = a_{21} b_{11} + a_{22} b_{21} + c_1$$
$$d_{22} = a_{21} b_{12} + a_{22} b_{22} + c_2$$

$$\Downarrow$$

$$\frac{\partial{L}}{\partial{b_{11}}} = \frac{\partial{L}}{\partial{d_{11}}} \frac{\partial{d_{11}}}{\partial{b_{11}}} + \frac{\partial{L}}{\partial{d_{21}}} \frac{\partial{d_{21}}}{\partial{b_{11}}} = \frac{\partial{L}}{\partial{d_{11}}} a_{11} + \frac{\partial{L}}{\partial{d_{21}}} a_{21}$$

$$\frac{\partial{L}}{\partial{b_{12}}} = \frac{\partial{L}}{\partial{d_{12}}} \frac{\partial{d_{12}}}{\partial{b_{12}}} + \frac{\partial{L}}{\partial{d_{22}}} \frac{\partial{d_{22}}}{\partial{b_{12}}} = \frac{\partial{L}}{\partial{d_{12}}} a_{11} + \frac{\partial{L}}{\partial{d_{22}}} a_{21}$$

$$\frac{\partial{L}}{\partial{b_{21}}} = \frac{\partial{L}}{\partial{d_{11}}} \frac{\partial{d_{11}}}{\partial{b_{21}}} + \frac{\partial{L}}{\partial{d_{21}}} \frac{\partial{d_{21}}}{\partial{b_{21}}} = \frac{\partial{L}}{\partial{d_{11}}} a_{12} + \frac{\partial{L}}{\partial{d_{21}}} a_{22}$$

$$\frac{\partial{L}}{\partial{b_{22}}} = \frac{\partial{L}}{\partial{d_{12}}} \frac{\partial{d_{12}}}{\partial{b_{22}}} + \frac{\partial{L}}{\partial{d_{22}}} \frac{\partial{d_{22}}}{\partial{b_{22}}} = \frac{\partial{L}}{\partial{d_{12}}} a_{12} + \frac{\partial{L}}{\partial{d_{22}}} a_{22}$$

$$\Downarrow$$

$$\begin{bmatrix}
\frac{\partial{L}}{\partial{b_{11}}} & \frac{\partial{L}}{\partial{b_{12}}}\\
\frac{\partial{L}}{\partial{b_{21}}} & \frac{\partial{L}}{\partial{b_{22}}}\\
\end{bmatrix} = \begin{bmatrix}
a_{11} & a_{21}\\
a_{12} & a_{22}\\
\end{bmatrix} \times \begin{bmatrix}
\frac{\partial{L}}{\partial{d_{11}}} & \frac{\partial{L}}{\partial{d_{12}}}\\
\frac{\partial{L}}{\partial{d_{21}}} & \frac{\partial{L}}{\partial{d_{22}}}\\
\end{bmatrix}
$$

$$\Downarrow$$

$$\frac{\partial{L}}{\partial{b}} = a^{T} \times \frac{\partial{L}}{\partial{d}}$$

In [47]:
# Computing "dloss/dW2"
dW2 = h.T @ dlogits

# Comparing the gradient computed
cmp('W2', dW2, W2)

W2              | exact: True  | approximate: True  | maxdiff: 0.0


#### `db2`

Lastly, we can do that for the bias term:

$$d_{11} = a_{11} b_{11} + a_{12} b_{21} + c_1$$
$$d_{12} = a_{11} b_{12} + a_{12} b_{22} + c_2$$
$$d_{21} = a_{21} b_{11} + a_{22} b_{21} + c_1$$
$$d_{22} = a_{21} b_{12} + a_{22} b_{22} + c_2$$

$$\Downarrow$$

$$\frac{\partial{L}}{\partial{c_{1}}} = \frac{\partial{L}}{\partial{d_{11}}} \frac{\partial{d_{11}}}{\partial{c_{1}}} + \frac{\partial{L}}{\partial{d_{21}}} \frac{\partial{d_{21}}}{\partial{c_{1}}} = \frac{\partial{L}}{\partial{d_{11}}} + \frac{\partial{L}}{\partial{d_{21}}}$$

$$\frac{\partial{L}}{\partial{c_{2}}} = \frac{\partial{L}}{\partial{d_{12}}} \frac{\partial{d_{12}}}{\partial{c_{2}}} + \frac{\partial{L}}{\partial{d_{22}}} \frac{\partial{d_{22}}}{\partial{c_{2}}} = \frac{\partial{L}}{\partial{d_{12}}} + \frac{\partial{L}}{\partial{d_{22}}}$$

$$\Downarrow$$

$$\frac{\partial{L}}{\partial{d}} = \begin{bmatrix}
\frac{\partial{L}}{\partial{d_{11}}} & \frac{\partial{L}}{\partial{d_{12}}}\\
\frac{\partial{L}}{\partial{d_{21}}} & \frac{\partial{L}}{\partial{d_{22}}}\\
\end{bmatrix}$$

$$\frac{\partial{L}}{\partial{c}} = \begin{bmatrix}
\frac{\partial{L}}{\partial{d_{11}}} + \frac{\partial{L}}{\partial{d_{21}}}\\
\frac{\partial{L}}{\partial{d_{12}}} + \frac{\partial{L}}{\partial{d_{22}}}\\
\end{bmatrix}$$

We can seen here that the derivatives with respect to $c$ just represent the elements of loss function gradient with respect to logits where these are summed vertically. 

In [48]:
# Computing "dloss/db2"
db2 = dlogits.sum(0)

# Comparing the gradient computed
cmp('b2', db2, b2)

b2              | exact: True  | approximate: True  | maxdiff: 0.0


## Tanh-layer

```python
h = torch.tanh(hpreact)
```

Next, we move to the Tanh layer. The tanh function looks as follows:

$$\tanh(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$$

The derivative of such a function takes a complex form which is not really efficient for the backpropagation. However, it is possible to rewrite the derivative as follows:

$$\frac{\partial{\tanh(x)}}{\partial{x}} = \frac{(e^{x} + e^{-x}) (e^{x} + e^{-x}) - (e^{x} - e^{-x}) (e^{x} - e^{-x})}{(e^{x} + e^{-x})^2}$$

$$\frac{\partial{\tanh(x)}}{\partial{x}} = \frac{(e^{x} + e^{-x})^2 - (e^{x} - e^{-x})^2}{(e^{x} + e^{-x})^2}$$

$$\frac{\partial{\tanh(x)}}{\partial{x}} = 1 - \frac{(e^{x} - e^{-x})^2}{(e^{x} + e^{-x})^2}$$

$$\frac{\partial{\tanh(x)}}{\partial{x}} = 1 - \left(\frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}\right)^2$$

$$\Downarrow$$

$$\frac{\partial{\tanh(x)}}{\partial{x}} = 1 - \tanh^2(x)$$

In other words, we can already use the output of the hyperbolic tangent function to easily compute the derivative. `h` already stores the result of tanh function so we can use it:

In [49]:
# Computing "dloss/dhpreact"
dhpreact = (1.0 - h**2) * dh

# Comparing the gradient computed
cmp('hpreact', dhpreact, hpreact)

hpreact         | exact: True  | approximate: True  | maxdiff: 0.0


## Batch normalization layer

We have reached the batch normalization layer:

### `dbngain`

```python
hpreact = bngain * bnraw + bnbias
```

In [50]:
hpreact.shape, bngain.shape, bnraw.shape, bnbias.shape

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

`bngain` is replicated across the rows of `bnraw` so we have to consider the vertical sums:

In [51]:
# Computing "dloss/dbngain"
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)

# Comparing the gradient computed
cmp('bngain', dbngain, bngain)

bngain          | exact: True  | approximate: True  | maxdiff: 0.0


### `dbnraw`

```python
hpreact = bngain * bnraw + bnbias
```

In [52]:
hpreact.shape, bngain.shape, bnraw.shape, bnbias.shape

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

In [53]:
# Computing "dloss/dbnraw"
dbnraw = bngain * dhpreact

# Comparing the gradient computed
cmp('bnraw', dbnraw, bnraw)

bnraw           | exact: True  | approximate: True  | maxdiff: 0.0


### `dbnbias`

```python
hpreact = bngain * bnraw + bnbias
```

In [54]:
hpreact.shape, bngain.shape, bnraw.shape, bnbias.shape

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

In [55]:
# Computing "dloss/dbnbias"
dbnbias = dhpreact.sum(0, keepdim=True)

# Comparing the gradient computed
cmp('bnbias', dbnbias, bnbias)

bnbias          | exact: True  | approximate: True  | maxdiff: 0.0


### `dbndiff` (first node)

Now we have to be careful, since `bndiff` occurs in two nodes. Let's consider the first one:

```python
bnraw = bndiff * bnvar_inv
```

In [56]:
bnraw.shape, bndiff.shape, bnvar_inv.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [57]:
# Computing "dloss/dbndiff" (Node 1)
dbndiff = bnvar_inv * dbnraw

# Comparing the gradient computed
cmp('bndiff', dbndiff, bndiff)

bndiff          | exact: False | approximate: False | maxdiff: 0.001157917082309723


### `dbnvar_inv`

```python
bnraw = bndiff * bnvar_inv
```

In [58]:
bnraw.shape, bndiff.shape, bnvar_inv.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [59]:
# Computing "dloss/dbnvar_inv"
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

# Comparing the gradient computed
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

bnvar_inv       | exact: True  | approximate: True  | maxdiff: 0.0


### `dbnvar`

```python
bnvar_inv = (bnvar + 1e-5)**-0.5
```

In [60]:
bnvar_inv.shape, bnvar.shape

(torch.Size([1, 64]), torch.Size([1, 64]))

In [61]:
# Computing "dloss/dbnvar"
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv

# Comparing the gradient computed
cmp('bnvar_inv', dbnvar, bnvar)

bnvar_inv       | exact: True  | approximate: True  | maxdiff: 0.0


### `dbndiff2`

```python
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
```

In [62]:
bnvar.shape, bndiff.shape

(torch.Size([1, 64]), torch.Size([32, 64]))

Let's use a toy example to depict what the above operation achieves:

$$A = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}$$

$$\Downarrow$$

$$A^{sum} = \frac{1}{2} \begin{bmatrix}
a_{11} + a_{21} + a_{31}\\
a_{12} + a_{22} + a_{32}\\
a_{13} + a_{23} + a_{33}
\end{bmatrix}$$

As can be notices, we are summing up the elements of $A$ across the rows and apply the scaling factor (Bessel correction) afterwards. The derivative of this is just a matrix of ones multiplied by chain rule derivative with scaling factor accounted for:

In [63]:
# Computing "dloss/dbndiff2"
dbndiff2 = (1.0 / (n - 1)) * torch.ones_like(bndiff2) * dbnvar

# Comparing the gradient computed
cmp('bndiff2', dbndiff2, bndiff2)

bndiff2         | exact: True  | approximate: True  | maxdiff: 0.0


### `dbndiff` (second node)

We have reached the second node where `bndiff` occurs:

```python
bndiff2 = bndiff**2
```

In [64]:
bndiff2.shape, bndiff.shape

(torch.Size([32, 64]), torch.Size([32, 64]))

In [65]:
# Accumulating "dloss/dbndiff" from Node 2
dbndiff += 2.0 * bndiff * dbndiff2

# Comapring the gradient computed
cmp('bndiff', dbndiff, bndiff)

bndiff          | exact: True  | approximate: True  | maxdiff: 0.0


### `dhprebn` (first node)

`dhprebn` also flows through two channels. The first node:

```python
bndiff = hprebn - bnmeani
```

In [66]:
bndiff.shape, hprebn.shape, bnmeani.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [67]:
# Computing "dloss/dhprebn" (Node 1)
dhprebn = dbndiff.clone()

# Comparing the gradient computed
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approximate: False | maxdiff: 0.0011137898545712233


### `dbnmeani`

```python
bndiff = hprebn - bnmeani
```

In [68]:
bndiff.shape, hprebn.shape, bnmeani.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [69]:
# Computing "dloss/dbnmeani"
dbnmeani = (-dbndiff).sum(0)

# Comparing the gradient computed
cmp('bnmeani', dbnmeani, bnmeani)

bnmeani         | exact: True  | approximate: True  | maxdiff: 0.0


### `dhprebn` (second node)

Here is the second node:

```python
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
```

In [70]:
bnmeani.shape, hprebn.shape

(torch.Size([1, 64]), torch.Size([32, 64]))

Let's consider a simpler example:

$$A = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}$$

$$\Downarrow$$

$$A^{sum} = \frac{1}{3} \begin{bmatrix}
a_{11} + a_{21} + a_{31}\\
a_{12} + a_{22} + a_{32}\\
a_{13} + a_{23} + a_{33}
\end{bmatrix}$$

In [71]:
# Accumulating "dloss/dhprebn" from Node 2
dhprebn += (1.0 / n) * torch.ones_like(hprebn) * dbnmeani

# Comparing the gradient computed
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: True  | approximate: True  | maxdiff: 0.0


## Linear layer 1

We have reached another linear layer and we can apply the above computed formulas:

```python
hprebn = embcat @ W1 + b1
```

In [72]:
# Computing "dloss/dembcat"
dembcat = dhprebn @ W1.T
# Computing "dloss/dW1"
dW1 = embcat.T @ dhprebn
# Computing "dloss/db1"
db1 = dhprebn.sum(0)

# Comparing the gradients computed
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)

embcat          | exact: True  | approximate: True  | maxdiff: 0.0
W1              | exact: True  | approximate: True  | maxdiff: 0.0
b1              | exact: True  | approximate: True  | maxdiff: 0.0


The next expression to get backpropagated through is as follows:

```python
embcat = emb.view(emb.shape[0], -1)
```

In [73]:
embcat.shape, emb.shape

(torch.Size([32, 30]), torch.Size([32, 3, 10]))

What this operation basically does is that we are just rearranging the elements of `emb` to obtain `embcat`. The derivative will just be the rearranged representation of `dembcat` from the chain rule in accordance with the dimensions of `emb`:

In [74]:
# Computing "dloss/demb"
demb = dembcat.view(emb.shape)

# Comparing the gradient computed
cmp('emb', demb, emb)

emb             | exact: True  | approximate: True  | maxdiff: 0.0


The last operation to consider is as follows:

```python
emb = C[Xb]
```

In [75]:
emb.shape, C.shape, Xb.shape

(torch.Size([32, 3, 10]), torch.Size([27, 10]), torch.Size([32, 3]))

In [76]:
Xb[:5]

tensor([[ 1,  1,  4],
        [18, 14,  1],
        [11,  5,  9],
        [ 0,  0,  1],
        [12, 15, 14]])

In [77]:
# Computing "dloss/dC"
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k, j]

# Verifying the gradient computed
cmp('C', dC, C)

C               | exact: True  | approximate: True  | maxdiff: 0.0


## Verification of all gradients

Lastly, we need to verify once again the correctness of all gradients we have manually computed:

In [78]:
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: